## Exploration_04. 작사가 인공지능 만들기 

### Step 1. 데이터 다운로드
* Cloud shell 에서 진행

### Step 2. 데이터 읽어오기

In [85]:
import glob
import os, re
import tensorflow as tf
from sklearn.model_selection import train_test_split # train, test set을 나누기 위함

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 읽어들인 txt 파일에 대한 정보를 모두 raw_corpus에 입력
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['I hear you callin\', "Here I come baby"', 'To save you, oh oh', "Baby no more stallin'"]


* 추후 사용을 위해 train_test_split 을 import 해주었다.
* 경로에 해당하는 text file들을 불러와 for 문에서 읽으면서 각 line에 대한 정보를 raw_corpus에 입력한다.

### Step 3. 데이터 정제

In [86]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 소문자로 통일, 공백제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip() # 공백제거
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 정상적으로 전처리가 되는지 테스트
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


* preprocess_sentence 함수에서는 입력받은 문장에 대한 전처리를 진행하게 된다.
* 전처리에서는 regular expression 을 이용해 문장내의 각종 특수문자들을 삭제하는 작업을 진행한다.
* 그리고 정제된 문장에 추후 사용 될 지표로써 'start', 'end' 를 추가 한 뒤 반환한다.
* 테스트 결과 문제없이 전처리가 된 것을 볼 수 있다.

In [87]:
# 정제된 문장 입력
corpus = []

for sentence in raw_corpus:
    # 공백만 있는 문장은 의미없는 공백이니 이를 제거한 뒤에 다음 문장으로 건너뜀
    if len(sentence.strip()) == 0: continue
    
    # 정제를 위한 함수 호출
    preprocessed_sentence = preprocess_sentence(sentence)
    
    # 15개 이상 단어인 문장은 제외
    if len(preprocessed_sentence.split()) > 15: continue
    
    # 위의 조건에 모두 부합하는 문장들만 corpus에 입력
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 확인
corpus[:5]

['<start> i hear you callin , here i come baby <end>',
 '<start> to save you , oh oh <end>',
 '<start> baby no more stallin <end>',
 '<start> these hands have been longing to touch you baby <end>',
 '<start> and now that you ve come around , to seein it my way <end>']

* 길이가 0인 문장, 끝이 ':'으로 끝나는 문장, 단어의 길이가 15개 이상인 문장들을 배제한 뒤, 조건을 모두 충족하는 문장들만 corpus에 입력하였다.
* 출력값을 확인해보면 예상대로 start, sentence, end 순으로 입력되어 있는 것을 알 수 있다.

In [88]:
# 텐서플로우의 Tokenizer와 pad_sequences를 사용해 tokenization 진행
def tokenize(corpus):

    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 12000 개의 tokenized 단어 생성
        filters=' ',      # preprecessing 과정을 이미 거쳤기 때문에 생략
        oov_token="<unk>" # 12000 단어에 없는 단어들은 '<unk>: unknown' 으로 표시
    )
    
    # 저장 된 tokenizer의 단어와 입력된 corpus의 단어를 비교
    tokenizer.fit_on_texts(corpus)
    
    # corpus에 저장된 단어들을 각각의 token(index) 값으로 변환한 뒤 tensor에 저장
    tensor = tokenizer.texts_to_sequences(corpus)   

    # 행렬 연산을 위해 길이를 맞춰야 하므로 빈 공간을 0 으로 채운다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    # 출력값 확인
    print(tensor,tokenizer)
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   4 186 ...   0   0   0]
 [  2  10 588 ...   0   0   0]
 [  2  52  41 ...   0   0   0]
 ...
 [  2   4  92 ...   0   0   0]
 [  2   9 156 ...   0   0   0]
 [  2 178  16 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fd7fefe3350>


* 단어 대신 tokenized 된 값들이 숫자의 형태로 입력되어있고, padding 기능을 이용해 뒷부분은 0으로 채워진 것 또한 확인 가능하다.
* 텐서플로우에서 제공하는 tokenization 방식을 이용해 단어와 index를 매칭 시킬 수 가 있다. 게다가 tokenization에 대한 설명을 찾아보면 단순 매칭을 넘어서서 단어와 특수문자들의 조합 같은 경우에도 단어만 골라서 인식할 수 있다고 한다. 즉, 'dog' 라는 형태와 'dog!' 라는 형태를 모두 'dog'라는 동일한 token으로 인식 한다는 것이다. 이런 발전이 유저들의 편의성을 상당히 높여주는 것이라 생각한다.

### Step 4. 평가 데이터셋 분리

In [89]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

# training data와 test data를 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=1)

# 확인용 출력
print(enc_train[0])
print(enc_val[0])
print(dec_train[0])
print(dec_val[0])

# train data의 shape 확인 (4-7. step4 의 예제와 동일한 수치가 나오는지 확인)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

[  2  45 899   1  19   8   3   0   0   0   0   0   0   0]
[  2   4  35 182   7  58   5  27  19 298   8  19 592   3]
[ 45 899   1  19   8   3   0   0   0   0   0   0   0   0]
[  4  35 182   7  58   5  27  19 298   8  19 592   3   0]
Source Train: (124960, 14)
Target Train: (124960, 14)


* source data 와 target data 중 20% 를 test 용으로 분리 하고 나머지를 training 에 사용한다.
* LMS의 '4-7. Step.4 평가 데이터셋 분리' 의 out 값과 동일하게 나오는지 확인. 결과적으로 동일하게 나오는 것으로 봐서 정상적으로 전처리 및 데이터 분리 과정들이 이뤄졌음을 알 수 있다.

In [91]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

# 미리 설정 된 값이 12000개 이고 거기에 pad 값으로 0이 추가 되기에 +1을 해 준다.
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)) # train data
dataset = dataset.shuffle(BUFFER_SIZE) # 전체 데이터를 셔플하기 위해 BUFFER_SIZE 설정
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) # drop_remainder=True 필수
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

* 먼저 from_tensor_slices()를 이용하여 입력된 값들을 tensorflow의 dataset형태로 저장한다. 이 때 앞서 분리해 둔 train data 를 데이터를 사용한다.
* 그 다음 dataset 전체를 shuffle해 준 뒤 BATCH_SIZE 만큼의 data를 가져온다. 이 때 drop_remainder의 값을 True로 설정해 주어야 항상 BATCH_SIZE 만큼의 데이터를 일정하게 가져오기 때문에, 이 점을 잊지 않도록 주의한다.


### Step 5. 인공지능 만들기

In [92]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        # embedding : index에 맞는 text를 matching
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

#### RNN vs LSTM
* 일반적인 RNN을 사용할 시에는 초기의 정보를 계속해서 기억하면서 끌고가기에 어려움이 있다. 즉, 초기에 계산 된 정보가 저 멀리있는 노드에서 사용되고자 할 때 이를 비교하고 수정하려먼 backpropagation으로 계산을 해야 되는데 거리가 멀기 때문에 다시 돌아오는 과정에서 gradient 값이 거의 0에 가깝게 줄어드는 문제가 있고 이를 vanishing gradient problem 이라고 한다. 이런 문제를 해결하기 위해 도입된 것이 Long-Short Term Memory(LSTM) network 이고, 따라서 위의 코드에서도 LSTM을 사용하고 있는 걸 볼 수 있다.

In [93]:
# Adam : optimizer중의 하나로 stochastic gradient descent (SGD) method를 base로 한다.
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10)

Epoch 1/10
488/488 [==============================] - 156s 311ms/step - loss: 4.0339
Epoch 2/10
488/488 [==============================] - 154s 315ms/step - loss: 3.0766
Epoch 3/10
488/488 [==============================] - 154s 315ms/step - loss: 2.8943
Epoch 4/10
488/488 [==============================] - 154s 315ms/step - loss: 2.7608
Epoch 5/10
488/488 [==============================] - 154s 314ms/step - loss: 2.6478
Epoch 6/10
488/488 [==============================] - 154s 315ms/step - loss: 2.5529
Epoch 7/10
488/488 [==============================] - 154s 315ms/step - loss: 2.4663
Epoch 8/10
488/488 [==============================] - 154s 315ms/step - loss: 2.3860
Epoch 9/10
488/488 [==============================] - 154s 315ms/step - loss: 2.3061
Epoch 10/10
488/488 [==============================] - 154s 315ms/step - loss: 2.2290


* epoch = 10 수행 결과  대략 30분정도 소요되며 loss는 2.2를 초과하여 나타났다. 따라서 2차로 epoch 횟수를 늘려 다시 진행해 볼 예정이다.

In [100]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = lyricist(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

#### 1차 결과 확인

In [98]:
# epoch = 10
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

* epoch = 10 : 굉장히 심플한 문장이 만들어졌다. 아마 text파일안에 i love you 라는 가사가 상당 수 들어있었던 것으로 짐작해 볼 수 있다.

In [99]:
# Adam : optimizer중의 하나로 stochastic gradient descent (SGD) method를 base로 한다.
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=20)

Epoch 1/20
488/488 [==============================] - 153s 309ms/step - loss: 2.2128
Epoch 2/20
488/488 [==============================] - 153s 313ms/step - loss: 2.1033
Epoch 3/20
488/488 [==============================] - 153s 313ms/step - loss: 2.0312
Epoch 4/20
488/488 [==============================] - 154s 316ms/step - loss: 1.9615
Epoch 5/20
488/488 [==============================] - 154s 315ms/step - loss: 1.8926
Epoch 6/20
488/488 [==============================] - 153s 313ms/step - loss: 1.8291
Epoch 7/20
488/488 [==============================] - 153s 312ms/step - loss: 1.7641
Epoch 8/20
488/488 [==============================] - 153s 313ms/step - loss: 1.6987
Epoch 9/20
488/488 [==============================] - 152s 312ms/step - loss: 1.6456
Epoch 10/20
488/488 [==============================] - 152s 312ms/step - loss: 1.5891
Epoch 11/20
488/488 [==============================] - 152s 312ms/step - loss: 1.5343
Epoch 12/20
488/488 [==============================] - 152s 311

#### 추가 학습 후 2차 결과 확인

In [101]:
# epoch = 20
generate_text(lyricist, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much <end> '

* epoch = 20 으로 진행한 결과, loss 는 1.1 정도로 떨어졌고 만들어지는 문장 또한 자연스러우며 좀 더 긴 문장이 생성되었다.

### 회고
* step 4 에서 source train과 target train의 수가 예제외 다르게 나오는 문제가 있었는데, step 3(In[87])의 공백만 존재하는 문장을 제거하는 과정에서 공백을 그대로 두고 넘어가면서 생긴 문제였다. 따라서 .strip()을 이용해 공백을 제거하므로써 이 문제를 해결할 수 있었다. 공백이 카운트 되는 경우들은 항시 조심할 필요가 있어보인다.
* 전체적인 tensorflow를 활용한 NLP의 기본적인 과정을 진행해 볼 수 있는 생소하지만 아주 좋은 경험이었다. tensorflow가 제공하는 다양한 기능에 확실히 익숙해질 필요가 있어보인다. 
* 테스트의 결과를 보자면 1차적으로 진행한 결과에서 매우 간단한 문장 'i love you'가 만들어 진걸로 봐서 좀 더 복잡한 형태의 문장을 작성하기에는 더 많은 학습과 loss를 줄이는 과정이 필요하다 생각되었다.
* 따라서 epoch 횟수를 20으로 늘리고 다시 진행해 본 결과 loss 수치가 절반가량(2.2 -> 1.1)으로 낮아 졌고, 생성된 문장 또한 문법에 어긋나지 않으면서 좀 더 긴 형태로 생성되었다. 따라서 추가적인 학습의 효과가 있다는 것을 알 수 있다.